N.B.: il seguente codice è stato utilizzato per le sedute 4, 5, 6, 8 e 9, ovvero per file in cui le colonne di cue negativi e speculativi, e dei relativi scope, non erano stati separati. Per farlo funzionare con eventuali sedute già annotate separando questi ultimi, bisognerà come minimo aggiustare gli indici delle colonne (quelli che si trovano in riga.split('\t)[NUMERO])

In [ ]:
import re
import copy

In [ ]:
numero_legislatura = 18
numero_resoconto = 9

In [ ]:
preannotato = f'Resoconti_annotati/legislatura_{numero_legislatura}/preannotati/{numero_resoconto}.tsv'

In [ ]:
resoconto_preann = []
with open(preannotato, 'r') as file:
    for line in file.readlines():
        resoconto_preann.append(line)

In [ ]:
annotato = f'Resoconti_annotati/legislatura_{numero_legislatura}/annotati/{numero_resoconto}.tsv'

In [ ]:
resoconto_ann = []
with open(annotato, 'r') as file:
    for line in file.readlines():
        resoconto_ann.append(line)

In [ ]:
# eliminazione delle tab aggiunte da Google Fogli/Excel (per il resoconto annotato)
for cnt,line in enumerate(resoconto_ann):
    if line[0] == '#':
        resoconto_ann[cnt] = re.sub(r'\t+','',line)
    if resoconto_ann[cnt].split('\t')[-1] == '\n':
        resoconto_ann[cnt] = resoconto_ann[cnt][:resoconto_ann[cnt].rfind('\t\n')] + '\n'

In [ ]:
resoconto_ann[0].split('\t')[11]

'CUE'

In [ ]:
# per ottenere gli indici delle righe in cui sono collocate le analisi delle frasi 
# (ed escludere invece le righe di commento che non sono il testo della frase)
indici_periodi = []
ultimo_numero = False
for cnt, line in enumerate(resoconto_ann):
    if cnt != 0: # per escludere la riga contenente la legenda delle colonne
        if line[0] != '#' and ultimo_numero == False:
            inizio_frase = cnt -1 #per salvare anche il commento con il testo
            ultimo_numero = True
        if line[0] == '#' and ultimo_numero == True:
            fine_frase = cnt-1
            indici_periodi.append((inizio_frase, fine_frase))
            ultimo_numero = False
        if cnt == len(resoconto_ann)-1:
            indici_periodi.append((inizio_frase, cnt))

In [ ]:
valori_cue = set()
for coppia_indici in indici_periodi:
    periodo = resoconto_ann[coppia_indici[0]+1:coppia_indici[1]+1]
    for riga in periodo:
        valori_cue.add(riga.split('\t')[11])
valori_cue

{'BN', 'BS', 'BS?', 'DESCR.', 'IS', 'IS?', 'O', '_'}

In [ ]:
pattern_numero = re.compile(r'[0-9]+')

In [ ]:
valori_scope = {'numero'}
for coppia_indici in indici_periodi:
    periodo = resoconto_ann[coppia_indici[0]+1:coppia_indici[1]+1]
    for riga in periodo:
        if len(pattern_numero.findall(riga.split('\t')[12])) == 0:
            valori_scope.add(riga.split('\t')[12])
valori_scope

{'DESCR.\n', '_', '_\n', '_?\n', 'numero'}

<h1>Per annotazione con editor di testo</h1>

In [ ]:
# per sostituire i valori = un solo spazio nelle colonne di CUE e SCOPE con un valore più facilmente leggibile da
# editor di testo
percorso_nuovo_file = f'Resoconti_annotati/legislatura_{numero_legislatura}/preannotati/{numero_resoconto}_mod.tsv'
with open(percorso_nuovo_file, 'w') as file:
    file.write(resoconto[0])
    for line in resoconto[1:]:
        if line[0] != '#':
            nuova_riga = line.replace(" ", 'SUBS')
            file.write(nuova_riga)
        else:
            file.write(line)

<h1> Per verificare il contenuto delle frasi negative o epistemizzate </h1>

In [ ]:
# per salvare solo le frasi contenenti una negazione o marca epistemica 
frasi_di_interesse = []
for coppia_indici in indici_periodi:
    lista_periodo = resoconto_ann[coppia_indici[0]:coppia_indici[1]+1]
    for riga in lista_periodo:
        if riga[0] != '#':
            cue = riga.split('\t')[11]
            if cue not in ('O','_','DESCR.'):
                frasi_di_interesse.append(lista_periodo)
                break

In [ ]:
frasi_di_interesse

[['# text = Se non vi sono osservazioni, il processo verbale si intende approvato.\n',
  '1\tSe\t_\t_\t_\t_\t4\tmark\t_\t_\t1\tO\t_\n',
  '2\tnon\t_\t_\t_\t_\t4\tadvmod\t_\t_\t2\tBN\t_\n',
  '3\tvi\t_\t_\t_\t_\t4\texpl\t_\t_\t3\tO\t2\n',
  '4\tsono\t_\t_\t_\t_\t11\tadvcl\t_\t_\t4\tO\t2\n',
  '5\tosservazioni\t_\t_\t_\t_\t4\tnsubj\t_\t_\t5\tO\t2\n',
  '6\t,\t_\t_\t_\t_\t4\tpunct\t_\t_\t6\tO\t_\n',
  '7\til\t_\t_\t_\t_\t8\tdet\t_\t_\t7\tO\t_\n',
  '8\tprocesso\t_\t_\t_\t_\t11\tnsubj:pass\t_\t_\t8\tO\t_\n',
  '9\tverbale\t_\t_\t_\t_\t8\tamod\t_\t_\t9\tO\t_\n',
  '10\tsi\t_\t_\t_\t_\t11\texpl:pass\t_\t_\t10\tO\t_\n',
  '11\tintende\t_\t_\t_\t_\t0\troot\t_\t_\t11\tO\t_\n',
  '12\tapprovato\t_\t_\t_\t_\t11\txcomp\t_\t_\t12\tO\t_\n',
  '13\t.\t_\t_\t_\t_\t11\tpunct\t_\t_\t13\tO\t_\n'],
 ["# text = Se non vi sono obiezioni, la proposta di istituzione della Commissione speciale e dell'attribuzione alla sua competenza dei predetti atti del Governo si intende accolta.\n",
  '1\tSe\t_\t_\t_\t_\t4\

<h1> Verifica resoconti annotati con Fogli </h1>

In [ ]:
# controllo uguaglianza numero di righe
len(resoconto_preann) == len(resoconto_ann)

NameError: name 'resoconto_preann' is not defined

In [ ]:
# controllo errori nello scope (che ogni scope corrisponda a un cue)
for coppia_indici in indici_periodi:
    periodo = resoconto_ann[coppia_indici[0]:coppia_indici[1]+1]
    lista_cue = []
    lista_scope = [] # contiene tuple: nel primo elemento l'ID del token nell'intervento, nel secondo il numero dello scope
    for riga in periodo[1:]:
        if 'B' in riga.split('\t')[11] or '?' in riga.split('\t')[11]:
            lista_cue.append(riga.split('\t')[10])
        if riga.split('\t')[12] not in ('_\n','DESCR.\n','_','DESCR.'):
            scope = riga.split('\t')[12].strip('\n')
            if ',' in scope:
                for numero in scope.split(','):
                    if '-' in numero:
                        lista_scope.append((riga.split('\t')[10],numero[:numero.find('-')].strip().strip('?')))
                    else:
                        lista_scope.append((riga.split('\t')[10],numero.strip().strip('?')))
            else:
                lista_scope.append((riga.split('\t')[10],scope.strip().strip('?')))
    for scope in lista_scope:
        if scope[1] not in lista_cue:
            print(f'Scope {scope[1]}, token id {scope[0]}, non trovato fra i cue per il seguente periodo: {periodo[0][periodo[0].find("=")+2:55]}')

In [ ]:
# verifica di non aver scordato una casella vuota
for coppia_indici in indici_periodi:
    periodo = resoconto_ann[coppia_indici[0]+1:coppia_indici[1]+1]
    for cnt,riga in enumerate(periodo):
        if len(riga.split('\t')) != 13:
            print("Numero riga: " + str(coppia_indici[0]+1+cnt))
            print(riga)

Numero riga: 4012
5	.	_	_	_	_	4	punct	_	_	12	O	_	


In [ ]:
# verifica uguaglianza riga per riga fra resoconti preannotati e annotati per assicurarsi dell'uguaglianza della codifica
for cnt, line in enumerate(resoconto_preann):
    if line[0]!='#':
        if line.split('\t')[:11] != resoconto_ann[cnt].split('\t')[:11]:
            print(f"Problema alle righe {cnt}, testi: \n{line} \nVS\n\n{resoconto_ann[cnt]}")
    else:
        if line != resoconto_ann[cnt]:
            print(f"Problema alle righe {cnt}, testi: \n{line} VS\n\n{resoconto_ann[cnt]}")
# se non stampa niente, le righe sono uguali (vengono ovviamente escluse le ultime due colonne, che sono quelle dell'annotazione)

In [ ]:
# confronto completo fra le righe (dovrebbero essere stampate solo quelle che differiscono per le ultime due colonne)
for cnt, line in enumerate(resoconto_preann):
    if line[0]!='#':
        if line.split('\t') != resoconto_ann[cnt].split('\t'):
            print(f"Problema alle righe {cnt}, testi: \n{line} \nVS\n\n{resoconto_ann[cnt]}")
    else:
        if line != resoconto_ann[cnt]:
            print(f"Problema alle righe {cnt}, testi: \n{line} VS\n\n{resoconto_ann[cnt]}")

Problema alle righe 56, testi: 
1	Se	_	_	_	_	4	mark	_	_	1	 	 
 
VS

1	Se	_	_	_	_	4	mark	_	_	1	O	_

Problema alle righe 57, testi: 
2	non	_	_	_	_	4	advmod	_	_	2	 	 
 
VS

2	non	_	_	_	_	4	advmod	_	_	2	BN	_

Problema alle righe 58, testi: 
3	vi	_	_	_	_	4	expl	_	_	3	 	 
 
VS

3	vi	_	_	_	_	4	expl	_	_	3	O	2

Problema alle righe 59, testi: 
4	sono	_	_	_	_	11	advcl	_	_	4	 	 
 
VS

4	sono	_	_	_	_	11	advcl	_	_	4	O	2

Problema alle righe 60, testi: 
5	osservazioni	_	_	_	_	4	nsubj	_	_	5	 	 
 
VS

5	osservazioni	_	_	_	_	4	nsubj	_	_	5	O	2

Problema alle righe 61, testi: 
6	,	_	_	_	_	4	punct	_	_	6	 	 
 
VS

6	,	_	_	_	_	4	punct	_	_	6	O	_

Problema alle righe 62, testi: 
7	il	_	_	_	_	8	det	_	_	7	 	 
 
VS

7	il	_	_	_	_	8	det	_	_	7	O	_

Problema alle righe 63, testi: 
8	processo	_	_	_	_	11	nsubj:pass	_	_	8	 	 
 
VS

8	processo	_	_	_	_	11	nsubj:pass	_	_	8	O	_

Problema alle righe 64, testi: 
9	verbale	_	_	_	_	8	amod	_	_	9	 	 
 
VS

9	verbale	_	_	_	_	8	amod	_	_	9	O	_

Problema alle righe 65, testi: 
10	si	

Problema alle righe 4883, testi: 
29	parlare	_	_	_	_	23	ccomp	_	_	1752	 	 
 
VS

29	parlare	_	_	_	_	23	ccomp	_	_	1752	O	1746

Problema alle righe 4884, testi: 
30	la	_	_	_	_	31	det	_	_	1753	 	 
 
VS

30	la	_	_	_	_	31	det	_	_	1753	O	1746

Problema alle righe 4885, testi: 
31	sentenza	_	_	_	_	29	nsubj	_	_	1754	 	 
 
VS

31	sentenza	_	_	_	_	29	nsubj	_	_	1754	O	1746

Problema alle righe 4886, testi: 
32	ultima	_	_	_	_	31	amod	_	_	1755	 	 
 
VS

32	ultima	_	_	_	_	31	amod	_	_	1755	O	1746

Problema alle righe 4888, testi: 
33	di	_	_	_	_	35	case	_	_	1756	 	 
 
VS

33	di	_	_	_	_	35	case	_	_	1756	O	1746

Problema alle righe 4889, testi: 
34	il	_	_	_	_	35	det	_	_	1757	 	 
 
VS

34	il	_	_	_	_	35	det	_	_	1757	O	1746

Problema alle righe 4890, testi: 
35	Consiglio	_	_	_	_	31	nmod	_	_	1758	 	 
 
VS

35	Consiglio	_	_	_	_	31	nmod	_	_	1758	O	1746

Problema alle righe 4891, testi: 
36	di	_	_	_	_	37	case	_	_	1759	 	 
 
VS

36	di	_	_	_	_	37	case	_	_	1759	O	1746

Problema alle righe 4892, testi: 
37	Stato	_

Problema alle righe 7811, testi: 
48	per	_	_	_	_	49	mark	_	_	1298	?	?
 
VS

48	per	_	_	_	_	49	mark	_	_	1298	O	1288

Problema alle righe 7812, testi: 
49	garantire	_	_	_	_	42	conj	_	_	1299	?	?
 
VS

49	garantire	_	_	_	_	42	conj	_	_	1299	O	1288

Problema alle righe 7814, testi: 
50	a	_	_	_	_	52	case	_	_	1300	?	?
 
VS

50	a	_	_	_	_	52	case	_	_	1300	O	1288

Problema alle righe 7815, testi: 
51	l'	_	_	_	_	52	det	_	_	1301	?	?
 
VS

51	l'	_	_	_	_	52	det	_	_	1301	O	1288

Problema alle righe 7816, testi: 
52	Autorità	_	_	_	_	49	obl	_	_	1302	?	?
 
VS

52	Autorità	_	_	_	_	49	obl	_	_	1302	O	1288

Problema alle righe 7817, testi: 
53	pieni	_	_	_	_	54	amod	_	_	1303	?	?
 
VS

53	pieni	_	_	_	_	54	amod	_	_	1303	O	1288

Problema alle righe 7818, testi: 
54	poteri	_	_	_	_	49	obj	_	_	1304	?	?
 
VS

54	poteri	_	_	_	_	49	obj	_	_	1304	O	1288

Problema alle righe 7819, testi: 
55	ma	_	_	_	_	57	cc	_	_	1305	?	?
 
VS

55	ma	_	_	_	_	57	cc	_	_	1305	O	_

Problema alle righe 7820, testi: 
56	per	_	_	_	_	57	mark	_	_	

Problema alle righe 11763, testi: 
22	,	_	_	_	_	25	punct	_	_	1355	 	 
 
VS

22	,	_	_	_	_	25	punct	_	_	1355	O	_

Problema alle righe 11764, testi: 
23	finalmente	_	_	_	_	25	advmod	_	_	1356	 	 
 
VS

23	finalmente	_	_	_	_	25	advmod	_	_	1356	O	_

Problema alle righe 11765, testi: 
24	,	_	_	_	_	23	punct	_	_	1357	 	 
 
VS

24	,	_	_	_	_	23	punct	_	_	1357	O	_

Problema alle righe 11766, testi: 
25	rispettare	_	_	_	_	20	conj	_	_	1358	 	 
 
VS

25	rispettare	_	_	_	_	20	conj	_	_	1358	O	1347

Problema alle righe 11767, testi: 
26	il	_	_	_	_	27	det	_	_	1359	 	 
 
VS

26	il	_	_	_	_	27	det	_	_	1359	O	1347

Problema alle righe 11768, testi: 
27	risultato	_	_	_	_	25	obj	_	_	1360	 	 
 
VS

27	risultato	_	_	_	_	25	obj	_	_	1360	O	_

Problema alle righe 11770, testi: 
28	di	_	_	_	_	30	case	_	_	1361	 	 
 
VS

28	di	_	_	_	_	30	case	_	_	1361	O	1347

Problema alle righe 11771, testi: 
29	l'	_	_	_	_	30	det	_	_	1362	 	 
 
VS

29	l'	_	_	_	_	30	det	_	_	1362	O	1347

Problema alle righe 11772, testi: 
30	esito	_	_	


Problema alle righe 14077, testi: 
17	di	_	_	_	_	18	case	_	_	1371	 	 
 
VS

17	di	_	_	_	_	18	case	_	_	1371	O	_

Problema alle righe 14078, testi: 
18	distribuzione	_	_	_	_	16	nmod	_	_	1372	 	 
 
VS

18	distribuzione	_	_	_	_	16	nmod	_	_	1372	O	_

Problema alle righe 14079, testi: 
19	,	_	_	_	_	16	punct	_	_	1373	 	 
 
VS

19	,	_	_	_	_	16	punct	_	_	1373	O	_

Problema alle righe 14080, testi: 
20	altro	_	_	_	_	21	det	_	_	1374	 	 
 
VS

20	altro	_	_	_	_	21	det	_	_	1374	O	_

Problema alle righe 14081, testi: 
21	tema	_	_	_	_	16	appos	_	_	1375	 	 
 
VS

21	tema	_	_	_	_	16	appos	_	_	1375	O	_

Problema alle righe 14082, testi: 
22	di	_	_	_	_	23	case	_	_	1376	 	 
 
VS

22	di	_	_	_	_	23	case	_	_	1376	O	_

Problema alle righe 14083, testi: 
23	interesse	_	_	_	_	21	nmod	_	_	1377	 	 
 
VS

23	interesse	_	_	_	_	21	nmod	_	_	1377	O	_

Problema alle righe 14084, testi: 
24	totale	_	_	_	_	23	amod	_	_	1378	 	 
 
VS

24	totale	_	_	_	_	23	amod	_	_	1378	O	_

Problema alle righe 14085, testi: 
25	per	_	_	_	_


Problema alle righe 16051, testi: 
6	anche	_	_	_	_	7	advmod	_	_	1053	 	 
 
VS

6	anche	_	_	_	_	7	advmod	_	_	1053	O	_

Problema alle righe 16052, testi: 
7	idee	_	_	_	_	5	obj	_	_	1054	 	 
 
VS

7	idee	_	_	_	_	5	obj	_	_	1054	O	_

Problema alle righe 16053, testi: 
8	diverse	_	_	_	_	7	amod	_	_	1055	 	 
 
VS

8	diverse	_	_	_	_	7	amod	_	_	1055	O	_

Problema alle righe 16054, testi: 
9	,	_	_	_	_	13	punct	_	_	1056	 	 
 
VS

9	,	_	_	_	_	13	punct	_	_	1056	O	_

Problema alle righe 16055, testi: 
10	quindi	_	_	_	_	13	cc	_	_	1057	 	 
 
VS

10	quindi	_	_	_	_	13	cc	_	_	1057	O	_

Problema alle righe 16056, testi: 
11	non	_	_	_	_	13	advmod	_	_	1058	 	 
 
VS

11	non	_	_	_	_	13	advmod	_	_	1058	BN	_

Problema alle righe 16057, testi: 
12	mi	_	_	_	_	13	iobj	_	_	1059	 	 
 
VS

12	mi	_	_	_	_	13	iobj	_	_	1059	O	1058

Problema alle righe 16058, testi: 
13	sembra	_	_	_	_	5	conj	_	_	1060	 	 
 
VS

13	sembra	_	_	_	_	5	conj	_	_	1060	BS	1058

Problema alle righe 16059, testi: 
14	neanche	_	_	_	_	13	advmod	_	_	106

<h1> Eliminazione colonna ID_TOKEN_NELL_INTERVENTO </h1>

In [ ]:
resoconto_ann[0].split('\t')[0]

'ID'

In [ ]:
resoconto_ann[0].split('\t')[10]

'ID_TOKEN_NELL_INTERVENTO'

In [ ]:
resoconto_ann[0].split('\t')[11]

'CUE'

In [ ]:
resoconto_ann[0].split('\t')[12]

'SCOPE\n'

Bisogna sostituire il numero/i numeri nella colonna scope, che identifica(no) il/i cue progressivamente nell'intervento, con l'id progressivo del cue nel periodo: questo perché fenomeni inter-periodo non vengono presi in considerazione.

In [ ]:
valori_cue

{'BN', 'BS', 'BS?', 'DESCR.', 'IS', 'IS?', 'O', '_'}

In [ ]:
valori_scope

{'DESCR.\n', '_', '_\n', '_?\n', 'numero'}

In [ ]:
for coppia_indici in indici_periodi:
    periodo = resoconto_ann[coppia_indici[0]+1:coppia_indici[1]+1]
    mapping_id = {}
    for riga in periodo:
        if 'B' in riga.split('\t')[11] or '?' in riga.split('\t')[11]:
            mapping_id[riga.split('\t')[10]] = riga.split('\t')[0]
    for cnt,riga in enumerate(periodo[:]):
        if riga.split('\t')[12] not in valori_scope:
            nuovi_scope = re.sub(pattern_numero, lambda x : mapping_id[x[0]],riga.split('\t')[12])
            nuova_riga = riga.split('\t')[:12]
            nuova_riga.append(nuovi_scope)
            nuova_riga = '\t'.join(nuova_riga)
            periodo[cnt] = nuova_riga + "\n"
    resoconto_ann[coppia_indici[0]+1:coppia_indici[1]+1] = periodo

E adesso procediamo all'eliminazione della colonna con l'id del token nell'intervento:

In [ ]:
for cnt, riga in enumerate(resoconto_ann[:]):
    if riga[0] != '#':
        resoconto_ann[cnt] = riga.split('\t')[:10]
        resoconto_ann[cnt].extend(riga.split('\t')[10+1:])
        resoconto_ann[cnt] = "\t".join(resoconto_ann[cnt])

<h1> Separazione cue e scope negazione e speculazione </h1>

In [ ]:
backup = copy.deepcopy(resoconto_ann)

In [ ]:
resoconto_ann[0]

'ID\tFORM\tLEMMA\tCPOS\tPOS\tFEATS\tHEAD\tDEPREL\tPHEAD\tPDEPREL\tCUE\tSCOPE\n'

In [ ]:
resoconto_ann[0].split('\t')[10]

'CUE'

In [ ]:
resoconto_ann[0].split('\t')[11]

'SCOPE\n'

In [ ]:
for coppia_indici in indici_periodi:
    periodo = resoconto_ann[coppia_indici[0]+1:coppia_indici[1]+1]
    for cnt,line in enumerate(periodo):
        if "-" in line.split('\t')[0]:
            resoconto_ann[coppia_indici[0]+1+cnt] = resoconto_ann[coppia_indici[0]+1+cnt].strip('\n') + "\t_\t_\n"
        elif line.split('\t')[10] == 'DESCR.':
            resoconto_ann[coppia_indici[0]+1+cnt] = resoconto_ann[coppia_indici[0]+1+cnt].strip('\n') + "\tDESCR.\tDESCR.\n"
        else:
            line = line.strip('\n')
            scope_neg = []
            scope_spec = []
            numeri_scope = line.split('\t')[11].split(',')
            for scope in numeri_scope:
                if '-' in scope:
                    scope_spec.append(scope[:scope.find('-')])
                elif re.search(pattern_numero,scope) is not None:
                    for line_2 in periodo:  #si poteva anche evitare di fare un doppio ciclo (perché prima controllo le righe che hanno il trattino,
                                            #che sono quelle che danno problema se usi direttamente il numero dello scope come indice) ma il codice 
                                            #sarebbe stato più difficile da comprendere e quindi mantenere
                        id_line_2 = line_2.split('\t')[0]
                        if str(id_line_2) == str(re.search(pattern_numero, scope)[0]):
                            cue_line_2 = line_2.split('\t')[10]
                            if 'BN' in cue_line_2:
                                scope_neg.append(scope)
                            elif 'BS' in cue_line_2:
                                scope_spec.append(scope)
                elif scope == '_?':
                    scope_neg.append(scope)
                    scope_spec.append(scope)
                    print("Scope incerto ('_?') assegnato a negazione e speculazione: correggere a mano. Coordinate:")
                    print("Periodo:" + resoconto_ann[coppia_indici[0]][resoconto_ann[coppia_indici[0]].find('= '):])
                    print("Token per scope incerto:" + line.split('\t')[1])
                    print()
                    print()
            cue_neg = "O"
            cue_spec = "O"
            valori_cue = line.split('\t')[10].split(',')
            for cue in valori_cue:
                if 'N' in cue:
                    cue_neg = cue
                elif 'S' in cue:
                    cue_spec = cue
                elif '?' in cue:
                    print("Periodo:" + resoconto_ann[coppia_indici[0]][resoconto_ann[coppia_indici[0]].find('= '):])
                    print("Token per cue incerto:" + line.split('\t')[1])
                    errore = True
                    while errore == True:
                        errore = False
                        risposta = input("Possibile cue negativo o speculativo?[n/s]")
                        if 'n' in risposta:
                            cue_neg = cue
                        elif 's' in risposta:
                            cue_spec = cue
                        else:
                            errore = True
                            print("Input errato. Riprova.")
                    print()
                    print()
            nuova_riga = line.split('\t')[:10]
            nuova_riga.append(cue_neg)
            if len(scope_neg)>0:
                scope_neg = ",".join(scope_neg)
            else:
                scope_neg = "_"
            nuova_riga.append(scope_neg)
            nuova_riga.append(cue_spec)
            if len(scope_spec)>0:
                scope_spec = ",".join(scope_spec)
            else:
                scope_spec = "_"
            nuova_riga.append(scope_spec)
            resoconto_ann[coppia_indici[0]+1+cnt] = "\t".join(nuova_riga) + "\n"

Scope incerto ('_?') assegnato a negazione e speculazione: correggere a mano. Coordinate:
Periodo:= Non fu questo, fu solo uno strumento per impedire la morte di un territorio.

Token per scope incerto:fu


Scope incerto ('_?') assegnato a negazione e speculazione: correggere a mano. Coordinate:
Periodo:= Nulla si ripete in modo uguale e i contesti sono certo diversi, però se non ricordiamo bene i giganti resteremo solo come nani come persone, come deputati, come forze politiche, come Paese (Applausi dei deputati dei gruppi Partito Democratico e Liberi e Uguali).

Token per scope incerto:ricordiamo


Scope incerto ('_?') assegnato a negazione e speculazione: correggere a mano. Coordinate:
Periodo:= Nulla si ripete in modo uguale e i contesti sono certo diversi, però se non ricordiamo bene i giganti resteremo solo come nani come persone, come deputati, come forze politiche, come Paese (Applausi dei deputati dei gruppi Partito Democratico e Liberi e Uguali).

Token per scope incerto:i




In [ ]:
lunghezze = set()
for line in resoconto_ann:
    if line[0] != '#':
        lunghezze.add(len(line.split('\t')))
lunghezze

{12, 14}

In [ ]:
for cnt,line in enumerate(resoconto_ann):
    if line[0] != '#':
        if len(line.split('\t'))==12:
            print(str(cnt) + ": "+line)

0: ID	FORM	LEMMA	CPOS	POS	FEATS	HEAD	DEPREL	PHEAD	PDEPREL	CUE	SCOPE



In [ ]:
resoconto_ann[0].split('\t')[10]

'CUE'

In [ ]:
resoconto_ann[0] = "\t".join(resoconto_ann[0].split('\t')[:10]) + "\tCUE_NEG\tSCOPE_NEG\tCUE_SPEC\tSCOPE_SPEC\n"

In [ ]:
resoconto_ann[0]

'ID\tFORM\tLEMMA\tCPOS\tPOS\tFEATS\tHEAD\tDEPREL\tPHEAD\tPDEPREL\tCUE_NEG\tSCOPE_NEG\tCUE_SPEC\tSCOPE_SPEC\n'

In [ ]:
lunghezze = set()
for line in resoconto_ann:
    lunghezze.add(len(line.split('\t')))
lunghezze

{1, 14}

Salvataggio del nuovo resoconto (il vecchio è comunque disponibile nella cartella Backup):

In [ ]:
with open(annotato, 'w') as file:
    for line in resoconto_ann:
        file.write(line)

<h1> Prova encoding </h1>

Ho creato un file contenente tutti i caratteri che possono occorrere nelle ultime due colonne (che sono quelle che mi riprendo dal file elaborato con Fogli Google), l'ho aperto in Fogli Google e poi l'ho salvato come .tsv. Il confronto dice che sono identici: no problem.

In [ ]:
with open('prova_encoding.tsv', 'r', encoding='utf-8') as file:
    python = file.read()

In [ ]:
with open('prova_encoding.tsv', 'r', encoding='utf-8') as file:
    fogli = file.read()

In [ ]:
python == fogli

True

In [ ]:
ord(fogli[0])

66

In [ ]:
ord(python[0])

66

In [ ]:
for cnt,character in enumerate(fogli):
    if ord(character) != ord(python[cnt]):
        print('Diverso!')

<h1> Calcolo numero token analizzati </h1>

In [ ]:
from pathlib import Path
import os
import re

In [ ]:
percorso_cartella = Path('Resoconti_annotati/legislatura_18/annotati')

In [ ]:
numero_token_analizzati = 0
for file in [c for c in os.scandir(percorso_cartella) if c.name[-4:] == '.tsv']:
    with open(file.path,'r') as file:
        riga = file.readline()
        while riga is not '':
            if riga[0]!='#' and '-' not in riga.split('\t')[0]:
                numero_token_analizzati += 1
            riga = file.readline()

In [ ]:
numero_token_analizzati

51820

In [ ]:
pattern_numero = re.compile(r'[0-9]+')
cue_neg = set()
cue_spec = set()
scope_neg = set()
scope_spec = set()
for file in [c for c in os.scandir(percorso_cartella) if c.name[-4:] == '.tsv']:
    with open(file.path,'r') as file:
        riga = file.readline()
        riga = file.readline()
        while riga is not '':
            if riga[0]!='#' and '-' not in riga.split('\t')[0]:
                cue_neg.add(riga.split('\t')[10])
                cue_spec.add(riga.split('\t')[12])
                if len(pattern_numero.findall(riga.split('\t')[11])) == 0:
                    scope_neg.add(riga.split('\t')[11])
                if len(pattern_numero.findall(riga.split('\t')[13])) == 0:
                    scope_spec.add(riga.split('\t')[13])
            riga = file.readline()

In [ ]:
cue_neg

{'BN', 'DESCR.', 'IN', 'O', 'O?'}

In [ ]:
cue_spec

{'BS', 'BS?', 'DESCR.', 'IS', 'IS?', 'O', 'O?'}

In [ ]:
scope_neg

{'DESCR.', '_', '_?'}

In [ ]:
scope_spec

{'DESCR.\n', '_\n', '_?\n'}

In [ ]:
no_token_rivisti_a_mano = 0 #limite inferiore: alcuni periodi li avrai rivisti e deciso che non contenevano negazioni o speculazioni
for file in [c for c in os.scandir(percorso_cartella) if c.name[-4:] == '.tsv']:
    with open(file.path,'r') as file:
        riga = file.readline()
        riga = file.readline()
        da_aggiungere = False
        while riga is not '':
            if '# text =' in riga:
                if da_aggiungere:
                    no_token_rivisti_a_mano += counter_temp
                counter_temp = 0
                da_aggiungere = False
            if riga[0]!='#' and '-' not in riga.split('\t')[0] and 'DESCR.' not in riga.split('\t')[10]:
                counter_temp += 1
                if da_aggiungere != True:
                    if 'B' in riga.split('\t')[10] or 'B' in riga.split('\t')[12]:
                        da_aggiungere = True
            riga = file.readline()
        if da_aggiungere == True:
            no_token_rivisti_a_mano += counter_temp

In [ ]:
no_token_rivisti_a_mano

21391